In [4]:
import random
import mysql.connector
import json
import string
import datetime as dt
from pymongo import MongoClient
from pprint import pprint

def getGenero(argument):
    switcher = {        
        5: "action",
        6: "adventure",
        7: "casual",
        8: "strategy",
        9: "RPG",
        10: "simulation",
        11: "early_access",
        12: "free_to_play",
        13: "sports",
        14: "racing",
        15: "massively_multiplayer"
    }
    return switcher.get(argument, argument)

myclient = MongoClient("mongodb://localhost:27017/")
mydbmongo = myclient['stum_for_you']
mycol = mydbmongo["usuarios"]
colmul = mydbmongo["multidimensional"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)
from datetime import datetime,timedelta
hoy = datetime.now()
dias = timedelta(days=7)
semana_pasada = hoy-dias  
#se comprueba que sea domingo (0)
comp=hoy.strftime("%w")
print(comp)
if (comp == '4') :#--------------------------------------------------------------cuidado
    
    hoy=hoy.strftime("%Y-%m-%d")
    #semana_pasada=semana_pasada.strftime("%Y:%m:%d")
    
    semana_pasada ="2017-01-01"#-------------------------------------------------para pruebas
    print(hoy,semana_pasada)
    mycursor = mydb.cursor(buffered=True)

    mycursor.execute("SELECT * FROM transacciones_venta WHERE fecha_venta BETWEEN %s AND  %s", (semana_pasada,hoy,))
    transacciones = mycursor.fetchall()
    tipo_movimiento = "venta"
    
    for t in transacciones :
        es_juego = False
        es_dlc = False
        existe_descuento = False
        existe_descuento_personalizado = False
        descuentoaplicado=0
    #"movimiento"
        precio_final = t[1]
        print(t[0])
        mycursor.execute("SELECT * FROM venta_juegos WHERE id_transaccion = %s", (t[0],))
        venta = mycursor.fetchall()
        cantidad = len(venta)
    #fecha
        anyo=t[2].strftime("%Y")
        mes=t[2].strftime("%m")
        dia=t[2].strftime("%d")
        fechasql = t[2].strftime("%Y-%m-%d") 
    #juego
        if(len(venta)>0) :
            es_juego = True
            mycursor.execute("SELECT id_juego FROM claves_juegos WHERE id_clave = %s", (venta[0][1],))
            puntero_juego = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM juegos WHERE id_juego = %s", (puntero_juego,))
            juego = mycursor.fetchone()
            genero= []
            #sacar genero
            for a in range (5,15) :
                if (juego[a]==True) :
                    genero.append(getGenero(a))
            precio_juego=juego[3]
            clasificacion=juego[1]
            #descuento_personalizado
            puntero_usuario = venta[0][2]
            descuento_personalizado = mycol.find_one({"id_usuario": puntero_usuario, "$or" :[{"cupon":{"tipo": "all"}},{"cupon":{"tipo": "dlc"}}]})
            if descuento_personalizado:
                existe_descuento_personalizado = True
                porcentaje_personalizado = descuento_personalizado["porcentaje"]
                fecha_inicio_personalizado = descuento_personalizado["fecha_inicio"]
                fecha_fin_personalizado = descuento_personalizado["fecha_fin"]
                tipo_personalizado = descuento_personalizado["tipo"]
                descuentoaplicado=porcentaje_personalizado
            else:
                #descuento 
                mycursor.execute("SELECT * FROM descuentos WHERE id_juego = %s and %s BETWEEN fecha_creacion and fecha_vencimiento ", (puntero_juego,fechasql,))
                descuento = mycursor.fetchone()
                if descuento:
                    existe_descuento = True
                    puntero_descuento=descuento[0]
                    porcentaje = descuento[1]
                    fecha_inicio = descuento[2].strftime("%Y-%m-%d")
                    fecha_fin = descuento[3].strftime("%Y-%m-%d")
                    descuentoaplicado=porcentaje

     #dlc
        if(len(venta)<=0) :
            es_dlc = True
            mycursor.execute("SELECT * FROM venta_dlcs WHERE id_tansaccion = %s", (t[0],))
            venta = mycursor.fetchall()
            cantidad = len(venta)
            mycursor.execute("SELECT id_dlc FROM claves_dlc WHERE id_clave = %s", (venta[0][0],))
            puntero_dlc = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM dlcs WHERE id_dlc = %s", (puntero_dlc,))
            dlc = mycursor.fetchone()
            tipo= dlc[3]
            precio_dlc=dlc[1]
            #descuento_personalizado
            puntero_usuario = venta[0][2]
            descuento_personalizado = mycol.find_one({"id_usuario": puntero_usuario, "$or" :[{"cupon":{"tipo": "all"}},{"cupon":{"tipo": "dlc"}}]})
            if descuento_personalizado:
                existe_descuento_personalizado = True
                porcentaje_personalizado = descuento_personalizado["porcentaje"]
                fecha_inicio_personalizado = descuento_personalizado["fecha_inicio"]
                fecha_fin_personalizado = descuento_personalizado["fecha_fin"]
                tipo_personalizado = descuento_personalizado["tipo"]
                descuentoaplicado=porcentaje_personalizado
            
            #descuento
            mycursor.execute("SELECT * FROM descuentos WHERE id_dlc = %s and %s BETWEEN fecha_creacion and fecha_vencimiento ", (puntero_dlc,fechasql,))
            descuento = mycursor.fetchone()
            if descuento:
                existe_descuento = True
                puntero_descuento=descuento[0]
                porcentaje = descuento[1]
                fecha_inicio = descuento[2].strftime("%Y-%m-%d")
                fecha_fin = descuento[3].strftime("%Y-%m-%d")
                descuentoaplicado=porcentaje
            
    #usuario
        puntero_usuario=venta[0][2]
        usuario = mycol.find_one({"id_usuario": puntero_usuario})
        sexo = usuario["sexo"]
        fechaNacimiento = usuario["fecha_nacimiento"]
        #ubicacion
        pais = usuario["pais"]
        ciudad = usuario["ciudad"]

        datos = {
            "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad,"descuentoAplicado":descuentoaplicado},
            "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
            "ubicacion":{"pais":pais,"ciudad":ciudad},
            "usuario":{"puntero_usuario":puntero_usuario,"sexo":sexo,"fechaNacimiento":fechaNacimiento,"pais":pais,"ciudad":ciudad},
            }
        if es_juego:
            datos["juego"] = {"puntero_juego":puntero_juego,"genero":genero,"precio_juego":precio_juego,"clasificacion":clasificacion}
        if es_dlc:
            datos["dlc"] = {"puntero_dlc":puntero_dlc,"precio_dlc":precio_dlc,"tipo":tipo}
        if existe_descuento:
            datos["descuento"] = {"puntero_descuento":puntero_descuento,"porcentaje":porcentaje,"fecha_inicio":fecha_inicio,"fecha_fin":fecha_fin}
        if existe_descuento_personalizado:
            datos["descuento_personalizado"] = {"porcentaje":porcentaje_personalizado,"fecha_inicio":fecha_inicio_personalizado,"fecha_fin":fecha_fin_personalizado, "tipo":tipo_personalizado}
        
        print("---------------------------------------------------------------")
        pprint(datos)
        x = colmul.insert_one(datos) 
    

4
2020-01-16 2017-01-01
2
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '03', 'mes': '06'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['adventure', 'simulation'],
           'precio': 15,
           'puntero_juego': 2117},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 15,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Alacant', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Alacant',
             'fechaNacimiento': '1999-09-13',
             'pais': 'Spain',
             'puntero_usuario': 1,
             'sexo': 'H'}}
4
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '07', 'mes': '07'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['RPG'],
           'precio': 3,
           'puntero_juego': 6063},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'p

             'puntero_usuario': 10,
             'sexo': 'H'}}
24
---------------------------------------------------------------
{'descuento': {'fecha_fin': '2019-10-11',
               'fecha_inicio': '2019-09-13',
               'porcentaje': 47,
               'puntero_descuento': 55},
 'fecha': {'anyo': '2019', 'dia': '10', 'mes': '10'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['adventure'],
           'precio': 7,
           'puntero_juego': 11048},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 47,
                'precio_final': 3,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Torrejón de Ardoz', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Torrejón de Ardoz',
             'fechaNacimiento': '1997-06-30',
             'pais': 'Spain',
             'puntero_usuario': 10,
             'sexo': 'H'}}
25
---------------------------------------------------------------
{'fecha': {'anyo': '2018', 'dia': '03', 'mes': '03'},
 '

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '28', 'mes': '10'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'adventure', 'casual', 'RPG'],
           'precio': 4,
           'puntero_juego': 9871},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 4,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Murcia', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Murcia',
             'fechaNacimiento': '1991-11-10',
             'pais': 'Spain',
             'puntero_usuario': 21,
             'sexo': 'H'}}
52
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '01', 'mes': '08'},
 'juego': {'clasificacion': 'AAA',
           'genero': [],
           'precio': 7,
           'puntero_juego': 7058},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 7,
 

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '06', 'mes': '04'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action'],
           'precio': 0,
           'puntero_juego': 8688},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 0,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Parla', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Parla',
             'fechaNacimiento': '2000-03-09',
             'pais': 'Spain',
             'puntero_usuario': 35,
             'sexo': 'M'}}
89
---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '18', 'mes': '03'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'strategy'],
           'precio': 5,
           'puntero_juego': 9768},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 5,
           

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '28', 'mes': '11'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['strategy'],
           'precio': 5,
           'puntero_juego': 8013},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 5,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Melilla', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Melilla',
             'fechaNacimiento': '1992-06-26',
             'pais': 'Spain',
             'puntero_usuario': 44,
             'sexo': 'M'}}
116
---------------------------------------------------------------
{'fecha': {'anyo': '2018', 'dia': '10', 'mes': '12'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'strategy', 'simulation'],
           'precio': 15,
           'puntero_juego': 756},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_fin

...

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '27', 'mes': '07'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['RPG'],
           'precio': 5,
           'puntero_juego': 5115},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 5,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Vienna', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Vienna',
             'fechaNacimiento': '1997-09-24',
             'pais': 'Austria',
             'puntero_usuario': 676,
             'sexo': 'M'}}
1711
---------------------------------------------------------------
{'fecha': {'anyo': '2017', 'dia': '14', 'mes': '08'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'RPG'],
           'precio': 13,
           'puntero_juego': 12202},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 13,
       

---------------------------------------------------------------
{'fecha': {'anyo': '2019', 'dia': '22', 'mes': '12'},
 'juego': {'clasificacion': 'AAA',
           'genero': ['adventure', 'casual'],
           'precio': 10,
           'puntero_juego': 7223},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 10,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Wörgl', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Wörgl',
             'fechaNacimiento': '1993-12-09',
             'pais': 'Austria',
             'puntero_usuario': 689,
             'sexo': 'M'}}
1742
---------------------------------------------------------------
{'fecha': {'anyo': '2017', 'dia': '23', 'mes': '08'},
 'juego': {'clasificacion': 'INDIE',
           'genero': ['action', 'adventure'],
           'precio': 2,
           'puntero_juego': 7254},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_f

---------------------------------------------------------------
{'dlc': {'precio': 15, 'puntero_dlc': 30, 'tipo': 'moneda virtual '},
 'fecha': {'anyo': '2019', 'dia': '12', 'mes': '05'},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 15,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Palma de Mallorca', 'pais': 'Spain'},
 'usuario': {'ciudad': 'Palma de Mallorca',
             'fechaNacimiento': '1999-06-30',
             'pais': 'Spain',
             'puntero_usuario': 4,
             'sexo': 'H'}}
1766
---------------------------------------------------------------
{'dlc': {'precio': 11, 'puntero_dlc': 214, 'tipo': ' trama adicional'},
 'fecha': {'anyo': '2019', 'dia': '03', 'mes': '04'},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 11,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Pontevedra', 'pais': 'Spain'},
 'usuario'

---------------------------------------------------------------
{'dlc': {'precio': 22, 'puntero_dlc': 323, 'tipo': ' cosmetico '},
 'fecha': {'anyo': '2019', 'dia': '26', 'mes': '09'},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 22,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Belfort', 'pais': 'France'},
 'usuario': {'ciudad': 'Belfort',
             'fechaNacimiento': '1994-09-12',
             'pais': 'France',
             'puntero_usuario': 365,
             'sexo': 'H'}}
1800
---------------------------------------------------------------
{'dlc': {'precio': 27, 'puntero_dlc': 466, 'tipo': 'moneda virtual '},
 'fecha': {'anyo': '2019', 'dia': '14', 'mes': '07'},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 27,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Creil', 'pais': 'France'},
 'usuario': {'ciudad': 'Creil',
  

---------------------------------------------------------------
{'dlc': {'precio': 27, 'puntero_dlc': 466, 'tipo': 'moneda virtual '},
 'fecha': {'anyo': '2019', 'dia': '14', 'mes': '07'},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 27,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Vienna', 'pais': 'Austria'},
 'usuario': {'ciudad': 'Vienna',
             'fechaNacimiento': '2000-07-23',
             'pais': 'Austria',
             'puntero_usuario': 630,
             'sexo': 'H'}}
1835
---------------------------------------------------------------
{'dlc': {'precio': 23, 'puntero_dlc': 596, 'tipo': ' trama adicional'},
 'fecha': {'anyo': '2019', 'dia': '06', 'mes': '04'},
 'movimiento': {'cantidad': 1,
                'descuentoAplicado': 0,
                'precio_final': 23,
                'tipo_movimiento': 'venta'},
 'ubicacion': {'ciudad': 'Eisenstadt', 'pais': 'Austria'},
 'usuario': {'ciudad': 